<a href="https://colab.research.google.com/github/Sirabhop/Preclinical-AD-EEG-classification/blob/master/EEG_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Library installation**


In [2]:
!pip install mne

     |████████████████████████████████| 6.6MB 2.9MB/s 


In [0]:
pip install git+https://github.com/raphaelvallat/entropy.git

  Cloning https://github.com/raphaelvallat/entropy.git to /tmp/pip-req-build-9wy21x0m
  Running command git clone -q https://github.com/raphaelvallat/entropy.git /tmp/pip-req-build-9wy21x0m
  Created wheel for entropy: filename=entropy-0.1.1-cp36-none-any.whl size=15459 sha256=22d9ff643f106139747465f59944cec22937c72fa51fea6272b9e31bfe2a3611
  Stored in directory: /tmp/pip-ephem-wheel-cache-zhqzwqc0/wheels/60/ed/d3/b715e38438f1f39edb1383aea79c578073953b25fa576fc71e
Successfully built entropy


In [45]:
pip install git+https://github.com/nice-tools/nice.git

  Cloning https://github.com/nice-tools/nice.git to /tmp/pip-req-build-pnaso2dh
  Running command git clone -q https://github.com/nice-tools/nice.git /tmp/pip-req-build-pnaso2dh
  Created wheel for nice: filename=nice-0.1.dev0-cp36-none-any.whl size=53945 sha256=a9bc03c4a91ce15aabb7b29096de44a7039e73df412535386a11930232aaa4e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ftpig46/wheels/50/a0/a3/2d473364ff7f9fdd3d41b0da4b4e34ef98c713cfa30695c2b0
Successfully built nice


# **Feature Extraction**

### **1) Power Spectral Density (PSD)** x5
*Delta (1-4Hz), Theta (4-8Hz), Alpha (8-12Hz), Beta (12-30Hz), Gamma (30-45Hz; Gaubert et al., 2019)*

In [0]:
def psd(data, method):
  from nice.markers.spectral import psd_welch
  import numpy as np

  psd_delta, freq_delta   = psd_welch(data, fmin = 0, fmax = 4)
  psd_theta, freq_theta   = psd_welch(data, fmin = 4, fmax = 8)
  psd_alpha, freq_alpha   = psd_welch(data, fmin = 8, fmax = 12)
  psd_beta, freq_beta     = psd_welch(data, fmin = 12, fmax = 30)
  psd_gamma, freq_gamma   = psd_welch(data, fmin = 30, fmax = 46)

  if method == 'mean':
    psds_m_d = np.mean(psd_delta)
    psds_m_th = np.mean(psd_theta)
    psds_m_al = np.mean(psd_alpha)
    psds_m_be = np.mean(psd_beta)
    psds_m_gm = np.mean(psd_gamma)
  elif method == 'median':
    psds_m_d = np.median(psd_delta)
    psds_m_th = np.median(psd_theta)
    psds_m_al = np.median(psd_alpha)
    psds_m_be = np.median(psd_beta)
    psds_m_gm = np.median(psd_gamma)
  psd_list = [psds_m_d, psds_m_th, psds_m_al, psds_m_be, psds_m_gm]
  return psd_list

### **2) Median Spectral Frequency (MSF)** x1

In [0]:
def MSF(data):
  from mne.time_frequency import psd_welch
  import numpy as np

  psd_total, freq_total = psd_welch(data, fmin = 0, fmax = 40)
  psd_m_total = np.median(psd_total)
  return psd_m_total

### **3) Spectral Entropy (SE)** x1


Using [Shannon entropy of the PSD of data (Inouye, T. et al. (1991)](https://raphaelvallat.com/entropy/build/html/generated/entropy.spectral_entropy.html#entropy.spectral_entropy)


In [0]:
def SpectralEntropy(data):
  import entropy
  import numpy as np

  #Convert mne object to dataframe
  #ch_names = ['Fp1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3',
   #         'O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8',
    #        'AF4','Fp2','Fz','Cz'] #32 electrodes
  df = data.to_data_frame(index = None)

  #Convert dataframe to array
  for x in data.info['ch_names']:
    array = np.array(df[x])
    array = np.concatenate([array, array])

  #Calculate entropy
  Spectral_Entropy = entropy.spectral_entropy(array, sf = 1000, method = 'welch')
  return Spectral_Entropy

### **4) Algorithmic Complexity (AC)** x1
Estimated using Kolmogorov Complexity

In [0]:
def AlgorithmicComplexity(data):
  from nice.algorithms.information_theory import epochs_compute_komplexity
  import math
  import pandas as pd
  
  #nbins = Number of bins to use for symbolic transformation
  #Only {0,1,2,3,4,5,6,7,8,9} so n = 10 -> the bit would be log2_10
  AC = epochs_compute_komplexity(data, nbins = math.log2(10))
  AC = pd.DataFrame(AC).mean().mean()
  return AC

### **5) Functional Connectivity (wSMI)** x2
 weighted mutual symbolic information (wSMI) were summarized by calculating the median value from each electrodes
1.   wSMI of theta 
2.   wSMI of alpha



In [0]:
def wSMI(data, method):
  from nice.algorithms.connectivity import epochs_compute_wsmi
  from mne import filter
  import mne
  import numpy as np

  #Filter data
  raw_theta = data
  raw_theta.filter(l_freq = 4, h_freq = 8)
  raw_alpha = data
  raw_alpha.filter(l_freq = 8, h_freq = 12)

  trigger = ('Status')
  events = mne.find_events(data, stim_channel=trigger)

  #Epoching
  data_theta = mne.Epochs(raw_theta, events , preload = True)
  data_alpha = mne.Epochs(raw_alpha, events , preload = True)

  #wSMI
  wsmi_t, smi_t, sym_t, count_t = epochs_compute_wsmi(data_theta, kernel = 3, tau = 21, method_params = {'bypass_csd': True})
  wsmi_a, smi_a, sym_a, count_a = epochs_compute_wsmi(data_alpha, kernel = 3, tau = 41, method_params = {'bypass_csd': True})
  
  #Method
  if method == 'mean':
    wsmi = [np.mean(wsmi_t), np.mean(wsmi_a)]
  elif method == 'median':
    wsmi = [np.median(wsmi_t), np.median(wsmi_a)]
  return wsmi

# **Buiding DataFrame**

In [0]:
def getDataFrame_bdf(subjectID):
  import mne

  EEG_feature_names = ['id', 'PSD_Delta', 'PSD_Theta', 'PSD_Alpha', 'PSD_Beta', 'PSD_Gamma', 'MSF', 'SE', 'AC', 'wSMI_Alpha', 'wSMI_Theta']
  df = pd.DataFrame(None, columns = EEG_feature_names)

  from mne.preprocessing import compute_proj_eog

  os.chdir('/content/drive/My Drive/EEG Data')
  
  eog_chs = ('Leye','Reye','UBlink','DBlink','LMast','RMast')
  trigger = ('Status')
  #Exclude; Gsr = Skin conductance, Resp = Respiration belt, Plet = plethismograph (Blood pressure, HR), Temp = thermometer
  exclude_chs = ('EXG7','EXG8','GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp') 
  subjects = ('SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8')
  chn_names = ['Fp1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3',
               'Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8',
               'FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']

  #Subject ID expected to be list
  for SS in subjectID:
    
    raw = mne.io.read_raw_bdf(SS+'.bdf', exclude = exclude_chs, eog = eog_chs, stim_channel = trigger, preload = True)
    
    #Filter
    raw.filter(l_freq = 0.5, h_freq = 45)
    raw.notch_filter(freqs = (50, 100))

    #Denosing SSP
    eog_projs, _ = compute_proj_eog(raw, n_eeg = 1, reject=None, no_proj=True)
    raw.add_proj(eog_projs)

    #Epoching
    events = mne.find_events(raw, stim_channel=trigger)
    epoch = mne.Epochs(raw, events, preload = True)

    #5 fetures
    features = {'id': None, 'PSD_Delta': None, 'PSD_Theta': None, 'PSD_Alpha': None, 'PSD_Beta': None,
                'PSD_Gamma': None, 'MSF': None, 'SE': None, 'AC': None, 'wSMI_Alpha': None, 'wSMI_Theta': None}    
    psd_total = psd(epoch, 'median')

    features['id'] = SS
    features['PSD_Delta'] = psd_total[0]
    features['PSD_Theta'] = psd_total[1]
    features['PSD_Alpha'] = psd_total[2]
    features['PSD_Beta'] = psd_total[3]
    features['PSD_Gamma'] = psd_total[4]
    features['MSF'] = MSF(epoch)
    features['SE'] = SpectralEntropy(epoch)
    features['AC'] = AlgorithmicComplexity(epoch)

    wSMI_total = wSMI(raw, 'mean')
    features['wSMI_Theta'] = wSMI_total[0]
    features['wSMI_Alpha'] = wSMI_total[1]

    #Build DataFrame
    df = df.append(features, ignore_index = True)
  return df

In [0]:
def getDataFrame_fif(subjectID):
  import mne

  EEG_feature_names = ['id', 'PSD_Delta', 'PSD_Theta', 'PSD_Alpha', 'PSD_Beta', 'PSD_Gamma', 'MSF', 'SE', 'AC', 'wSMI_Alpha', 'wSMI_Theta']
  df = pd.DataFrame(None, columns = EEG_feature_names)

  from mne.preprocessing import compute_proj_eog

  os.chdir('/content/drive/My Drive/EEG Data')

  #Subject ID expected to be list
  for SS in subjectID:
    raw = mne.io.read_raw_fif(SS+'.fif', preload = True)
    #Filter
    raw.filter(l_freq = 0.5, h_freq = 45)
    raw.notch_filter(freqs = (50, 100))

    #Denosing SSP
    eog_projs, _ = compute_proj_eog(raw, n_eeg = 1, reject=None, no_proj=True)
    raw.add_proj(eog_projs)

    #Epoching
    trigger = ('Status')
    events = mne.find_events(raw, stim_channel=trigger)
    epoch = mne.Epochs(raw, events , preload = True)

    #5 fetures
    features = {'id': None, 'PSD_Delta': None, 'PSD_Theta': None, 'PSD_Alpha': None, 'PSD_Beta': None,
                'PSD_Gamma': None, 'MSF': None, 'SE': None, 'AC': None, 'wSMI_Alpha': None, 'wSMI_Theta': None}    
    psd_total = psd(epoch, 'median')

    features['id'] = SS
    features['PSD_Delta'] = psd_total[0]
    features['PSD_Theta'] = psd_total[1]
    features['PSD_Alpha'] = psd_total[2]
    features['PSD_Beta'] = psd_total[3]
    features['PSD_Gamma'] = psd_total[4]
    features['MSF'] = MSF(epoch)
    features['SE'] = SpectralEntropy(epoch)
    features['AC'] = AlgorithmicComplexity(epoch)

    wSMI_total = wSMI(raw, 'mean')
    features['wSMI_Theta'] = wSMI_total[0]
    features['wSMI_Alpha'] = wSMI_total[1]

    #Build DataFrame
    df = df.append(features, ignore_index = True)
  return df